In [ ]:
import os

import numpy as np
import pandas as pd
from Bio import SeqIO
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout
from keras.models import Sequential, load_model
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split

In [ ]:
def containsOnly(seq, alphabet="ACDEFGHIKLMNPQRSTVWY"):
    for char in seq:
        if char not in alphabet: 
            return False
    return True


def encode_labels(df):
    idx, labels = pd.factorize(df, sort=True)
    encoded = to_categorical(idx)
    return encoded, labels


def encode_kmers(kmers, alphabet="ACDEFGHIKLMNPQRSTVWY", k=35):
    encoded_kmers = []
    for kmer in kmers:
        kmer = kmer[:k]    # Make sure to take expected length
        try:
            idx = [alphabet.index(aa) for aa in kmer]
            encoded_kmer = to_categorical(idx, len(alphabet))
            encoded_kmer = np.array(encoded_kmer.flatten(), dtype=np.int)
            encoded_kmers.append(encoded_kmer)
        except ValueError:  # in case of non_amino_acid letters
            print(kmer)
    return np.array(encoded_kmers)


def encode_datasets_PLS(file, k=35, sep="\t", header=0):
    df = pd.read_csv(file, sep=sep, header=header)
    x = encode_kmers(df.motif, k=k)
    y, labels = encode_labels(df.name)
    return x, y, labels


def create_model(xshape, yshape):
    model = Sequential()
    model.add(Dense(256, input_dim=xshape, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(yshape, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    return model


def PPR_motifs(accession, sequence, model, labels, bg="B", k=35):
    kmers = [sequence[i:i + k] for i in range(len(sequence) - k + 1)]
    encoded_kmers = encode_kmers(kmers, k=k)
    y_probs = model.predict(encoded_kmers)
    y_classes = y_probs.argmax(axis=-1)

    starts = np.where(labels[y_classes] != bg)
    cls = y_classes[starts]
    proba = [y_probs[i][y_classes[i]] for i in starts[0]]
    motif = [sequence[s:s + k] for s in starts[0]]
    d = {"accession": accession,
         "start": starts[0],
         "end": starts[0] + k,
         "name": labels[cls],
         "score": proba,
         "strand": "+",
         "motif": motif}

    df = pd.DataFrame(d,
                      columns=['accession', 'start', 'end', 'name', 'score', 'strand', 'motif'])

    return df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

def plot_cm(model, x, y, lables, bb=None):
    y_p = model.predict(x)
    y_p_classes = y_p.argmax(axis=-1)
    y_p_lables = np.array(labels[y_p_classes])
    
    y_classes = y.argmax(axis=-1)
    y_lable = np.array(labels[y_classes])
    cmatrix = pd.crosstab(y_lable, y_p_lables, 
                          rownames=["curatedLabel"], 
                          colnames=["Prediction"])
    if bb:
        cmatrix['B']['B'] = bb
    sns.heatmap(cmatrix,
                annot=True, fmt='',
                cmap=plt.cm.Blues)

In [ ]:
x, y, labels = encode_datasets_PLS("dataset/new/Ath_167_19k.tsv", k=35)

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.25, random_state=42)

assert x_train.shape[0] == y_train.shape[0]
assert x_valid.shape[0] == y_valid.shape[0]

xshape, yshape = x_train.shape[1], y_train.shape[1]

print("x_train.shape: \t{}".format(x_train.shape))
print("y_train.shape: \t{}".format(y_train.shape))
print("x_valid.shape: \t{}".format(x_valid.shape))
print("y_valid.shape: \t{}".format(y_valid.shape))

In [ ]:
weights = "./Model/Model_Epoch_{epoch:02d}.h5"

if not os.path.exists(os.path.dirname(weights)):
    os.mkdir(os.path.dirname(weights))

In [ ]:
### Train model
model = create_model(xshape, yshape)

check_pointer = ModelCheckpoint(filepath=weights, 
                                verbose=0, 
                                save_best_only=False)

early_stopping = EarlyStopping(monitor='loss',
                               min_delta=1e-5, 
                               patience=10, 
                               verbose=1, 
                               mode='auto')
        
model.fit(x_train, y_train, 
          batch_size=128,
          epochs=50, 
          validation_data=(x_valid, y_valid),
          callbacks=[early_stopping, check_pointer], 
          shuffle=True,
          verbose=1)

In [ ]:
### Load Trained model
model_bs0 = load_model("./Model/Model_Epoch_44.h5")

In [ ]:
score, acc = model_bs0.evaluate(x, y, verbose=0)
print('Test loss: {}'.format(score))
print('Test accuracy: {}'.format(acc))

In [ ]:
plot_cm(model_bs0, x, y, labels, bb=3000)

In [ ]:
import re

def motifs_ath(model, prefix):
    P = prefix + "_P.tsv"
    L = prefix + "_L.tsv"
    T = prefix + "_T.tsv"
    U = prefix + "_U.tsv"
    N = prefix + "_N.tsv"
    fasta="dataset/ath_167/Athaliana_167_TAIR10.protein.fa"
    
    with open("dataset/ath_167/p_ath_ppr.txt") as f1:
        ath_P = [line.rstrip() for line in f1]

    with open("dataset/ath_167/pls_ath_ppr.txt") as f2:
        ath_L = [line.rstrip() for line in f2]

    with open("dataset/ath_167/t_ath_tpr.txt") as f3:
        ath_T = [line.rstrip() for line in f3]

    with open("dataset/ath_167/pt_ath.txt") as f4:
        ath_U = [line.rstrip() for line in f4]

    for record in SeqIO.parse(fasta, "fasta"):
        sequence = str(record.seq)[:-1] # remove "*" at the end of every sequence
        accession = record.id
    
        if len(sequence) < 35 or not containsOnly(sequence):
            continue
        #ATG = r"(AT.?G\d{5})"
        ATG = r"(AT.?G\d{5}\.1)"
        if re.search(ATG, accession):
            match = re.search(ATG, accession)
            acc = match.group()
            try:
                df = PPR_motifs(accession, sequence, model, labels, bg="B", k=35)
            except ValueError:
                continue
        else:
            continue
        if acc[:9] in ath_P:
            df.to_csv(P, mode='a', sep="\t", header=False, index=False)
        elif acc[:9] in ath_L:
            df.to_csv(L, mode='a', sep="\t", header=False, index=False)
        elif acc[:9] in ath_T:
            df.to_csv(T, mode='a', sep="\t", header=False, index=False)
        elif acc[:9] in ath_U:
            df.to_csv(U, mode='a', sep="\t", header=False, index=False)
        else:
            df.to_csv(N, mode='a', sep="\t", header=False, index=False)
    return None

In [ ]:
motifs_ath(model_bs0, prefix="dataset/ath_167/000")